In [ ]:
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(sys.path[0]), "..")))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
from torch import nn
from copy import deepcopy

from src.data_handler import Im2LatexDataHandler
from src.data_loader import get_data_loaders
from src.train import get_model
from src.analysis import MulticlassAccuracy

In [4]:
def evaluate(loader, model, tokenizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    acc = MulticlassAccuracy(
        num_classes=tokenizer.get_vocab_size(),
        ignore_index=-1,
        average="micro",
    ).to(device)
    
    i = 0
    for images, x, y in loader:
        images = images.to(device)
        x = x.to(device)
        y = y.to(device)

        logits, _ = model(x, images, y)

        # ==== Analysis ====

        # accuracy
        _, predicted = torch.max(logits, dim=-1)
        acc.update(predicted.view(-1), y.view(-1))

        if i % 100 == 0:
            print(
                f"Analysis: Iter: {i}, "
                f"Accuracy: {acc.compute():.4f}"
            )

        i += 1

    print(
        f"Analysis: Accuracy: {acc.compute():.4f}"
    )

In [5]:
model, tokenizer = get_model(
    "convnext_custom",
    "gpt",
    {"out_channel": 64},
    {"dropout": 0.0, "n_layer": 12, "n_head": 8},
    {"file_path": "./data/dataset5/step2/dict_id2word.pkl"},
)
model.load_state_dict(
    torch.load("./models/latexify-convnext_custom-64-gpt.pth", map_location="cpu")
)

Number of parameters in total: 0.90M
  - Encoder: 0.22M
  - Decoder: 0.68M


<All keys matched successfully>

In [6]:
handler = Im2LatexDataHandler(data_dir="./data/dataset5", train_percentage=0)
df_combined, y_combined, tuple_len = handler.load_data_and_images()

14280 14280
14280 14280


In [7]:
loader = get_data_loaders(
    df_combined,
    y_combined,
    "test",
    tokenizer,
    tuple_len,
    batch_size=16,
)

### Performance of the original model

In [8]:
evaluate(loader, model, tokenizer)

/home/hice1/yqiu343/.local/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Analysis: Iter: 0, Accuracy: 0.6582
Analysis: Iter: 100, Accuracy: 0.6294
Analysis: Iter: 200, Accuracy: 0.6281
Analysis: Iter: 300, Accuracy: 0.6286
Analysis: Iter: 400, Accuracy: 0.6287
Analysis: Iter: 500, Accuracy: 0.6288
Analysis: Iter: 600, Accuracy: 0.6292
Analysis: Iter: 700, Accuracy: 0.6291
Analysis: Iter: 800, Accuracy: 0.6292
Analysis: Accuracy: 0.6294


### Remove 1 layer from GPT

In [9]:
nmodel = deepcopy(model)
nmodel.decoder.transformer["h"][11] = nn.Identity()  # type: ignore

In [10]:
evaluate(loader, nmodel, tokenizer)

Analysis: Iter: 0, Accuracy: 0.5573
Analysis: Iter: 100, Accuracy: 0.5469
Analysis: Iter: 200, Accuracy: 0.5480
Analysis: Iter: 300, Accuracy: 0.5486
Analysis: Iter: 400, Accuracy: 0.5491
Analysis: Iter: 500, Accuracy: 0.5490
Analysis: Iter: 600, Accuracy: 0.5482
Analysis: Iter: 700, Accuracy: 0.5481
Analysis: Iter: 800, Accuracy: 0.5478
Analysis: Accuracy: 0.5481


### Remove 2 layers from GPT

In [11]:
nnmodel = deepcopy(nmodel)
nnmodel.decoder.transformer["h"][10] = nn.Identity()  # type: ignore

In [12]:
evaluate(loader, nnmodel, tokenizer)

Analysis: Iter: 0, Accuracy: 0.5201
Analysis: Iter: 100, Accuracy: 0.4775
Analysis: Iter: 200, Accuracy: 0.4776
Analysis: Iter: 300, Accuracy: 0.4773
Analysis: Iter: 400, Accuracy: 0.4769
Analysis: Iter: 500, Accuracy: 0.4777
Analysis: Iter: 600, Accuracy: 0.4775
Analysis: Iter: 700, Accuracy: 0.4779
Analysis: Iter: 800, Accuracy: 0.4784
Analysis: Accuracy: 0.4789


### Remove 1 CNBlock from ConvNeXt

In [13]:
nmodel = deepcopy(model)
nmodel.encoder.features[1][2] = nn.Identity()

In [14]:
evaluate(loader, nmodel, tokenizer)

Analysis: Iter: 0, Accuracy: 0.6343
Analysis: Iter: 100, Accuracy: 0.6292
Analysis: Iter: 200, Accuracy: 0.6280
Analysis: Iter: 300, Accuracy: 0.6276
Analysis: Iter: 400, Accuracy: 0.6284
Analysis: Iter: 500, Accuracy: 0.6295
Analysis: Iter: 600, Accuracy: 0.6298
Analysis: Iter: 700, Accuracy: 0.6299
Analysis: Iter: 800, Accuracy: 0.6298
Analysis: Accuracy: 0.6294


### Remove 2 CNBlocks from ConvNeXt

In [15]:
nnmodel = deepcopy(nmodel)
nnmodel.encoder.features[1][1] = nn.Identity()

In [16]:
evaluate(loader, nnmodel, tokenizer)

Analysis: Iter: 0, Accuracy: 0.6919
Analysis: Iter: 100, Accuracy: 0.6322
Analysis: Iter: 200, Accuracy: 0.6312
Analysis: Iter: 300, Accuracy: 0.6294
Analysis: Iter: 400, Accuracy: 0.6273
Analysis: Iter: 500, Accuracy: 0.6270
Analysis: Iter: 600, Accuracy: 0.6278
Analysis: Iter: 700, Accuracy: 0.6286
Analysis: Iter: 800, Accuracy: 0.6292
Analysis: Accuracy: 0.6294
